In [53]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
#from transformers import GPT2Tokenizer
import string
from torch.nn import functional as F
import numpy as np

In [54]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

In [55]:
df = pd.read_csv('/home/tom/Python/Machine learning/pytorch/lstm/files/XAU_USD Historical Data.csv' , thousands=',')

df = df[['Date', 'Open']]
# Convert 'Date' to datetime and sort by date
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values('Date', inplace=True)
df

,Date,Open
5993,2000-12-25,274.00
5992,2000-12-26,273.40
5991,2000-12-27,274.55
5990,2000-12-28,275.25
5989,2000-12-29,272.45
...,...,...
4,2023-12-26,2055.73
3,2023-12-27,2066.70
2,2023-12-28,2077.34
1,2023-12-29,2066.82


In [56]:
vocab_size = df['Open'].nunique()

In [57]:
open_values = torch.tensor(df['Open'].values.astype(np.float32)).view(-1).long()
print(open_values.shape, open_values.dtype)

torch.Size([5994]) torch.int64


In [58]:
# Train and test splits
n = int(0.9*len(open_values)) # first 90% will be train, rest val data
train_data = open_values[:n]
val_data = open_values[n:]
train_data.shape

torch.Size([5394])

In [59]:
def get_batch(split):
    # Generate a small batch of data with inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix]).to(device)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]).to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [60]:
X , Y = get_batch(train_data)
X.shape

torch.Size([16, 32])

In [61]:
class Head(nn.Module):
    """One head of self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)   # (B, T, C)
        q = self.query(x) # (B, T, C)

        # Compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * C**-0.5  # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))  # (B, T, T)
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        wei = self.dropout(wei)

        # Perform the weighted aggregation of the values
        v = self.value(x)  # (B, T, C)
        out = wei @ v  # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """Multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    """A simple linear layer followed by a non-linearity"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """Transformer block: communication followed by computation"""

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# Super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # Each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)  # Final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B, T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T, C)
        x = tok_emb + pos_emb  # (B, T, C)
        x = self.blocks(x)  # (B, T, C)
        x = self.ln_f(x)  # (B, T, C)
        logits = self.lm_head(x)  # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # Crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # Get the predictions
            logits, loss = self(idx_cond)
            # Focus only on the last time step
            logits = logits[:, -1, :]  # Becomes (B, C)
            # Apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # Append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

In [62]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

0.913811 M parameters
step 0: train loss 8.7800, val loss 8.8093
step 100: train loss 6.9001, val loss 9.2589
step 200: train loss 5.7120, val loss 9.1746
step 300: train loss 4.1133, val loss 9.0375
step 400: train loss 2.7689, val loss 9.3730
step 500: train loss 1.9449, val loss 9.8738
step 600: train loss 1.4415, val loss 10.1541
step 700: train loss 1.0684, val loss 10.6645
step 800: train loss 0.8284, val loss 10.8275
step 900: train loss 0.6476, val loss 11.1216
step 1000: train loss 0.5308, val loss 11.4122
step 1100: train loss 0.4492, val loss 11.6726
step 1200: train loss 0.4087, val loss 11.8729
step 1300: train loss 0.3630, val loss 11.9393
step 1400: train loss 0.3481, val loss 12.2605
step 1500: train loss 0.3149, val loss 12.0779
step 1600: train loss 0.2839, val loss 12.4909
step 1700: train loss 0.2743, val loss 12.5246
step 1800: train loss 0.2632, val loss 12.7763
step 1900: train loss 0.2539, val loss 12.7153
step 2000: train loss 0.2382, val loss 12.9804
step 2100

In [63]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(m.generate(context, max_new_tokens=2000)[0].tolist())

[0, 1383, 1393, 1396, 1380, 1371, 1376, 1385, 1385, 1380, 1384, 1383, 1405, 1411, 1406, 1421, 1414, 1380, 1377, 1372, 1369, 1374, 1381, 1388, 1373, 1359, 1361, 1368, 1370, 1345, 1343, 1334, 1333, 1346, 1312, 1336, 1332, 1340, 1336, 1353, 1348, 1350, 1364, 1363, 1363, 1359, 1362, 1373, 1374, 1383, 1388, 1406, 1399, 1411, 1402, 1408, 1410, 1434, 1435, 1416, 1428, 1431, 1428, 1429, 1412, 1422, 1429, 1396, 1401, 1405, 1426, 1427, 1429, 1439, 1432, 1427, 1422, 1418, 1424, 1432, 1428, 1437, 1456, 1460, 1460, 1473, 1462, 1452, 1459, 1476, 1486, 1497, 1497, 1503, 1507, 1511, 1505, 1506, 1527, 1536, 1567, 1545, 1537, 1516, 1474, 1493, 1506, 1493, 1491, 1486, 1498, 1495, 1512, 1518, 1527, 1524, 1523, 1537, 1539, 1535, 1542, 1535, 1544, 1543, 1545, 1538, 1545, 1530, 1516, 1523, 1530, 1528, 1538, 1540, 1548, 1549, 1522, 1500, 1496, 1501, 1512, 1502, 1486, 1496, 1517, 1530, 1533, 1544, 1555, 1566, 1582, 1588, 1595, 1605, 1589, 1598, 1592, 1599, 1614, 1618, 1613, 1616, 1628, 1620, 1659, 1661, 1648, 